# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [1]:
import requests
import time
import re
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

In [17]:
KEYWORDS = ['IT', 'python', 'парсинг', 'SQL']

In [18]:
# Функция формирования списка ссылок
def list_creation(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    time.sleep(0.3)

    links = []

    for post in posts:
        url = post.find('a', class_='post__title_link')
        a = url.attrs.get('href')
        links.append(a)
    return links

In [19]:
list_creation('https://habr.com/ru/all/')

['https://habr.com/ru/post/512848/',
 'https://habr.com/ru/company/vivaldi/blog/513880/',
 'https://habr.com/ru/company/microsoft/blog/513522/',
 'https://habr.com/ru/company/megafon/blog/513862/',
 'https://habr.com/ru/company/ua-hosting/blog/513688/',
 'https://habr.com/ru/company/vdsina/blog/513498/',
 'https://habr.com/ru/company/gazpromneft/blog/513796/',
 'https://habr.com/ru/post/513874/',
 'https://habr.com/ru/post/513366/',
 'https://habr.com/ru/post/513870/',
 'https://habr.com/ru/post/513440/',
 'https://habr.com/ru/post/513868/',
 'https://habr.com/ru/post/491978/',
 'https://habr.com/ru/post/513866/',
 'https://habr.com/ru/post/513860/',
 'https://habr.com/ru/post/513858/',
 'https://habr.com/ru/post/513854/',
 'https://habr.com/ru/company/lamptest/blog/513838/',
 'https://habr.com/ru/company/southbridge/blog/513790/',
 'https://habr.com/ru/post/513818/']

In [20]:
def create_info(links_list):

    list_ = []

    for link in links_list:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'html.parser')
        news = soup.find_all('div', class_='post__text')
        time.sleep(0.3)
    
        for el in news:
            el_lower = el.text.lower()  
            # ищем вхождение ключевого слова в посте
            for words in KEYWORDS:
                if re.findall(r'\b' + words.lower() + r'\b', el_lower):
                    data = soup.find('span', class_='post__time').attrs.get('data-time_published')
                    post_title = soup.find('span', class_='post__title-text').text
                    href = link
                    text = soup.find('div', class_='post__text').text
                    row = {'data': data, 'title': post_title, 'href': href, 'text': text} 
                    list_.append(row) 
                    break   
                
    df = pd.DataFrame(list_)
    df['data'] = df['data'].astype('datetime64')
    return df

In [21]:
%%time
create_info(list_creation('https://habr.com/ru/all/'))

Wall time: 37.5 s


,data,title,href,text
0,2020-08-05 07:05:00,Как проанализировать рынок фотостудий с помощь...,https://habr.com/ru/post/512848/,"Каждый, кто открывает свой бизнес, хочет угада..."
1,2020-08-05 06:30:00,Рынок соискателя или работодателя VS возрастна...,https://habr.com/ru/company/vdsina/blog/513498/,\n\r\nВ продолжение моего разбора полётов по ...
2,2020-08-05 06:17:00,"Корпоративный архитектор: похож на обычного, т...",https://habr.com/ru/company/gazpromneft/blog/5...,"Мало кто понимает, чем занимаются корпоративны..."
3,2020-08-05 05:06:00,Создание собственной Headless CMS и интеграция...,https://habr.com/ru/post/513366/,\nБыть новичком — значит исследовать новые гор...
4,2020-08-04 15:56:00,Завязывайте со своим «Хабр не торт». Хабр — эт...,https://habr.com/ru/post/513818/,"\n\r\nКогда я работал в офисе, я не очень пони..."


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [22]:
email_list = ['eterin@agriculture.ru',
         'lawpro-707@list.ru',
         'solowey011@yandex.ru',
         'ahds@aol.com',
         'akas@aol.com',
         'workpost@fxt.ru'
]

In [23]:
def check_email_list(email_list):
    list_of_leaks = []

    for email in email_list:
        url = 'https://digibody.avast.com/v1/web/leaks'
        params = {'email': email}
        req = requests.post(url, json=params)
        time.sleep(0.3)
        leaks = req.json()['value']

        for leak in leaks:
            email_2 = email
            date = leak['leak_info']['date']
            source = leak['leak_info']['title']
            description = leak['leak_info']['description'] 
            row = {'email': email_2, 'date': date, 'source': source, 'description': description} 
            list_of_leaks.append(row)
    df = pd.DataFrame(list_of_leaks)   
    return df      

In [24]:
table = check_email_list(email_list)

In [25]:
table.head()

,email,date,source,description
0,lawpro-707@list.ru,1565222400000,Borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
1,lawpro-707@list.ru,1588204800000,Payad,"At an unconfirmed date, the advertising revenu..."
2,solowey011@yandex.ru,1549411200000,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
3,solowey011@yandex.ru,1548720000000,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
4,solowey011@yandex.ru,1549411200000,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."


In [26]:
def data_conversion(data):
    d = datetime.fromtimestamp(int(str(data)[0:10])).strftime('%Y-%m-%d')
    return d

In [27]:
table['date'] = table['date'].apply(data_conversion)

In [28]:
table.head()

,email,date,source,description
0,lawpro-707@list.ru,2019-08-08,Borealis.su,"At an unconfirmed date, Russian Minecraft serv..."
1,lawpro-707@list.ru,2020-04-30,Payad,"At an unconfirmed date, the advertising revenu..."
2,solowey011@yandex.ru,2019-02-06,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
3,solowey011@yandex.ru,2019-01-29,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
4,solowey011@yandex.ru,2019-02-06,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [34]:
URL = 'https://api.vk.com/method/newsfeed.search?'
GROUP = 'netology'
TOKEN = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
VERSION = '5.103'

SLEEP = 0.33

In [35]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'q': GROUP,
    'count': 50
}

In [36]:
res = requests.get(URL, params)
# res.text
# res.json()

In [37]:
df_2 = pd.DataFrame(res.json()['response']['items'])
df_2.head()

,id,date,owner_id,from_id,post_type,text,attachments,comments,likes,reposts,views,is_favorite,marked_as_ads,signer_id
0,6805,1596611577,225949324,225949324,post,#ссылки@lib_vsmu \n\nКак организовать и провес...,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,NaN,NaN
1,5149,1596611520,-43895456,-43895456,post,"Курс: Дата-журналист: истории, основанные на д...","[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 1},False,0.0,NaN
2,414,1596610805,-172926588,-172926588,post,"""Как я ошибался"" \n\nНа ошибках учатся. Вот то...","[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,0.0,NaN
3,109,1596546003,-196278567,-196278567,post,У тебя еще нет планов на завтра? \n\nТогда обя...,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 15},False,0.0,NaN
4,130,1596545097,-183178935,-183178935,post,На очереди новая подборка из 5 полезных онлайн...,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 1},{'count': 1},{'count': 41},False,0.0,NaN


In [38]:
df_2['date'] = df_2['date'].apply(data_conversion)
df_2.head()

,id,date,owner_id,from_id,post_type,text,attachments,comments,likes,reposts,views,is_favorite,marked_as_ads,signer_id
0,6805,2020-08-05,225949324,225949324,post,#ссылки@lib_vsmu \n\nКак организовать и провес...,"[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,NaN,NaN
1,5149,2020-08-05,-43895456,-43895456,post,"Курс: Дата-журналист: истории, основанные на д...","[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 1},False,0.0,NaN
2,414,2020-08-05,-172926588,-172926588,post,"""Как я ошибался"" \n\nНа ошибках учатся. Вот то...","[{'type': 'link', 'link': {'url': 'https://net...",{'count': 0},{'count': 0},{'count': 0},{'count': 2},False,0.0,NaN
3,109,2020-08-04,-196278567,-196278567,post,У тебя еще нет планов на завтра? \n\nТогда обя...,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 0},{'count': 0},{'count': 15},False,0.0,NaN
4,130,2020-08-04,-183178935,-183178935,post,На очереди новая подборка из 5 полезных онлайн...,"[{'type': 'photo', 'photo': {'album_id': -7, '...",{'count': 0},{'count': 1},{'count': 1},{'count': 41},False,0.0,NaN


In [39]:
news_netology = df_2[['date', 'text']]
news_netology.head()

,date,text
0,2020-08-05,#ссылки@lib_vsmu \n\nКак организовать и провес...
1,2020-08-05,"Курс: Дата-журналист: истории, основанные на д..."
2,2020-08-05,"""Как я ошибался"" \n\nНа ошибках учатся. Вот то..."
3,2020-08-04,У тебя еще нет планов на завтра? \n\nТогда обя...
4,2020-08-04,На очереди новая подборка из 5 полезных онлайн...
